In [50]:
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import operator
import scipy
import math

import collections
from collections import Counter

import numpy as np
import scipy as sp
from  operator import itemgetter
from scipy.stats import norm
import seaborn as sb


In [143]:

week_1_dates = ["23-04-21", "24-04-21", "25-04-21"] 
week_2_dates = ["26-04-21", "27-04-21", "28-04-21", "29-04-21", "30-04-21", "01-05-21", "02-05-21"]
week_3_dates = ["03-05-21", "04-05-21", "05-05-21", "06-05-21", "07-05-21", "08-05-21", "09-05-21"]
week_4_dates = ["10-05-21"]


dir_dataset = 'datasetAprileMaggio/'
files_dataset1 = ['ln_' + str(date) + '.json' for date in week_1_dates]
files_dataset2 = ['ln_' + str(date) + '.json' for date in week_2_dates]
files_dataset3 = ['ln_' + str(date) + '.json' for date in week_3_dates]

dir_channels = 'datasetAprileMaggio/channels/'


In [146]:
dir_dataset = 'datasetAprileMaggio/unione_channels/'
files = ['week1.json','week2.json','week3.json','week4.json']


In [148]:
with open (dir_dataset + 'week1.json') as one, open (dir_dataset + 'week2.json') as two, open (dir_dataset + 'week3.json') as three, open (dir_dataset + 'week4.json') as four:
    data1 = json.load(one)
    data2 = json.load(two)
    data3 = json.load(three)
    data4 = json.load(four)
df1 = pd.json_normalize(data1)
df2 = pd.json_normalize(data2)
df3 = pd.json_normalize(data3)
df4 = pd.json_normalize(data4)


in_common12 = df1[df1['channel_id'].isin(df2['channel_id'])]
in_common13 = df1[df1['channel_id'].isin(df3['channel_id'])]
in_common14 = df1[df1['channel_id'].isin(df4['channel_id'])]
in_common23 = df2[df2['channel_id'].isin(df3['channel_id'])]
in_common24 = df2[df2['channel_id'].isin(df4['channel_id'])]
in_common34 = df3[df3['channel_id'].isin(df4['channel_id'])]

    
print("In comune tra week1 e week2:" ,len(in_common12))
print("In comune tra week1 e week3:" ,len(in_common13))
print("In comune tra week1 e week4:" ,len(in_common14))
print("In comune tra week2 e week3:" ,len(in_common23))
print("In comune tra week2 e week4:" ,len(in_common24))
print("In comune tra week3 e week4:" ,len(in_common34))

in_commonall = df1[df1['channel_id'].isin(df2['channel_id']) & df1['channel_id'].isin(df3['channel_id']) & df1['channel_id'].isin(df4['channel_id'])]
print("Canali in comune tra week1,week2,week3,week4:" , len(in_commonall))

In comune tra week1 e week2: 43823
In comune tra week1 e week3: 42563
In comune tra week1 e week4: 36762
In comune tra week2 e week3: 42563
In comune tra week2 e week4: 36762
In comune tra week3 e week4: 36762
Canali in comune tra week1,week2,week3,week4: 36762


In [49]:
to_write = in_commonall.to_json(orient='records', indent = 1)
with open(dir_dataset + 'channels_union.json', "w") as f:
    f.write(to_write)

In [149]:
#elimino i canali doppi : da multigrafo a grafo : ottengo 3mila canali in meno circa
df_without_duplicates = in_commonall.drop_duplicates(subset=['node1_pub','node2_pub'])
print(f"Canali in comune tra week1,week2,week3,week4 - dopo aver eliminato i multielementi: {len(df_without_duplicates)}\n")
df_without_dup_copy = df_without_duplicates.copy()
#rimuovo le colonne policy che sono sempre null
df_without_dup_copy = df_without_duplicates.drop(['node1_policy','node2_policy'],axis=1)
#elimino le righe in cui almeno un elemento è null -> df definitivo
df_without_dup_copy.dropna( how='any',axis=0,inplace=True)
print(f"Canali in comune tra week1,week2,week3,week4 CONFRONTABILI (senza campi null): {len(df_without_dup_copy)}\n")


Canali in comune tra week1,week2,week3,week4 - dopo aver eliminato i multielementi: 33470

Canali in comune tra week1,week2,week3,week4 CONFRONTABILI (senza campi null): 26340



In [152]:
#c e k contengono: (nodo,num_canali_uscenti_da_esso)
c = collections.Counter(df_without_dup_copy['node1_pub'])
k = collections.Counter(df_without_dup_copy['node2_pub'])
#z contiene l'unione di c e k: se lo stesso nodo compare in entrambi allora viene sommato il numero di canali uscenti
z = dict(c+k)
print(f"Nodi in comune tra week1,week2,week3,week4: {len(z)}\n")

#display(len(c))
#display(len(k))
#display(len(z))

#ordino z in ordine decrescente in base al numero di canali uscenti dal nodo
sorted_z = sorted(z.items(), key=lambda x: x[1], reverse=True)
display(sorted_z)


Nodi in comune tra week1,week2,week3,week4: 7549



[('02ad6fb8d693dc1e4569bcedefadf5f72a931ae027dc0f0c544b34c1c6f3b9a02b', 1101),
 ('0242a4ae0c5bef18048fbecf995094b74bfb0f7391418d71ed394784373f41e4f3', 933),
 ('0217890e3aad8d35bc054f43acc00084b25229ecff0ab68debd82883ad65ee8266', 881),
 ('0331f80652fb840239df8dc99205792bba2e559a05469915804c08420230e23c7c', 699),
 ('023d70f2f76d283c6c4e58109ee3a2816eb9d8feb40b23d62469060a2b2867b77f', 683),
 ('02004c625d622245606a1ea2c1c69cfb4516b703b47945a3647713c05fe4aaeb1c', 562),
 ('03bb88ccc444534da7b5b64b4f7b15e1eccb18e102db0e400d4b9cfe93763aa26d', 497),
 ('03abf6f44c355dec0d5aa155bdbdd6e0c8fefe318eff402de65c6eb2e1be55dc3e', 478),
 ('02315fe3619ffdea2561bcacecada87b226723f471a59fdbfec18c4e84bcf785b2', 449),
 ('0297a77f4d1ccc55d7a10a9b137119b1103d9a9d38a5a97ffa1d0152c818fcdd0a', 397),
 ('03e691f81f08c56fa876cc4ef5c9e8b727bd682cf35605be25d48607a802526053', 390),
 ('0390b5d4492dc2f5318e5233ab2cebf6d48914881a33ef6a9c6bcdbb433ad986d0', 339),
 ('0298f6074a454a1f5345cb2a7c6f9fce206cd0bf675d177cdbf0ca7508dd

In [25]:
nod1 = df_without_dup_copy['node1_pub'].values.tolist()
nod2 = df_without_dup_copy['node2_pub'].values.tolist()
nod = nod1 + nod2
fee_b1 = df_without_dup_copy['node1_policy.fee_base_msat'].values.tolist()
fee_b2 = df_without_dup_copy['node2_policy.fee_base_msat'].values.tolist()
fee = fee_b1 + fee_b2
len(nod)

52680

In [91]:

dict_w1 = {}

for node in z:

    dict_w1[node] = {'HTLC': [], 'FEE_BASE': [], 'FEE_RATE': []}

    dict_w1[node]['FEE_BASE'] = df1[df1.node1_pub.eq(node) & df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_base_msat'].values.tolist() + df1[df1.node2_pub.eq(node) & df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_base_msat'].values.tolist()
    dict_w1[node]['FEE_RATE'] = df1[df1.node1_pub.eq(node) & df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_rate_milli_msat'].values.tolist() + df1[df1.node2_pub.eq(node) & df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_rate_milli_msat'].values.tolist()

In [156]:
#dict_w1,_w2,_w3 e _w4 sono dizionari in cui per ogni nodo memorizzo 3 liste separate contenenti il valore delle fee applicate da esso nei canali uscenti 
#lo uso per trovare quante sono le commissioni che variano nelle settimane 

dict_w1 = {}

for node in z:
    dict_w1[node] = {'HTLC': [], 'FEE_BASE': [], 'FEE_RATE': []}

    dict_w1[node]['HTLC'] = df1[df1.node1_pub.eq(node) & df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.min_htlc'].values.tolist() + df1[df1.node2_pub.eq(node) & df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.min_htlc'].values.tolist()
    dict_w1[node]['FEE_BASE'] = df1[df1.node1_pub.eq(node) & df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_base_msat'].values.tolist() + df1[df1.node2_pub.eq(node) & df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_base_msat'].values.tolist()
    dict_w1[node]['FEE_RATE'] = df1[df1.node1_pub.eq(node) & df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_rate_milli_msat'].values.tolist() + df1[df1.node2_pub.eq(node) & df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_rate_milli_msat'].values.tolist()

dict_w2 = {}

for node in z:
    dict_w2[node] = {'HTLC': [], 'FEE_BASE': [], 'FEE_RATE': []}

    dict_w2[node]['HTLC'] = df2[df2.node1_pub.eq(node) & df2['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.min_htlc'].values.tolist() + df2[df2.node2_pub.eq(node) & df2['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.min_htlc'].values.tolist()
    dict_w2[node]['FEE_BASE'] = df2[df2.node1_pub.eq(node) & df2['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_base_msat'].values.tolist() + df2[df2.node2_pub.eq(node) & df2['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_base_msat'].values.tolist()
    dict_w2[node]['FEE_RATE'] = df2[df2.node1_pub.eq(node) & df2['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_rate_milli_msat'].values.tolist() + df2[df2.node2_pub.eq(node) & df2['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_rate_milli_msat'].values.tolist()

dict_w3 = {}

for node in z:
    dict_w3[node] = {'HTLC': [], 'FEE_BASE': [], 'FEE_RATE': []}

    dict_w3[node]['HTLC'] = df3[df3.node1_pub.eq(node) & df3['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.min_htlc'].values.tolist() + df3[df3.node2_pub.eq(node) & df3['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.min_htlc'].values.tolist()
    dict_w3[node]['FEE_BASE'] = df3[df3.node1_pub.eq(node) & df3['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_base_msat'].values.tolist() + df3[df3.node2_pub.eq(node) & df3['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_base_msat'].values.tolist()
    dict_w3[node]['FEE_RATE'] = df3[df3.node1_pub.eq(node) & df3['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_rate_milli_msat'].values.tolist() + df3[df3.node2_pub.eq(node) & df3['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_rate_milli_msat'].values.tolist()

dict_w4 = {}

for node in z:
    dict_w4[node] = {'HTLC': [], 'FEE_BASE': [], 'FEE_RATE': []}
    
    dict_w4[node]['HTLC'] = df4[df4.node1_pub.eq(node) & df4['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.min_htlc'].values.tolist() + df4[df4.node2_pub.eq(node) & df4['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.min_htlc'].values.tolist()
    dict_w4[node]['FEE_BASE'] = df4[df4.node1_pub.eq(node) & df4['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_base_msat'].values.tolist() + df4[df4.node2_pub.eq(node) & df4['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_base_msat'].values.tolist()
    dict_w4[node]['FEE_RATE'] = df4[df4.node1_pub.eq(node) & df4['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_rate_milli_msat'].values.tolist() + df4[df4.node2_pub.eq(node) & df3['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_rate_milli_msat'].values.tolist()

In [141]:
#dizionario in cui per ogni nodo memorizzo le fee uguali in ogni settimana
#la len delle liste indica su quanti canali il nodo ha deciso di lasciare invariate le fee nel corso delle settimane
 dict_incommon = {} 

for node in z:
    dict_incommon[node] = {'HTLC':[], 'FEE_BASE':[], 'FEE_RATE':[]}

    dict_incommon[node]['HTLC'] = [i for i, j, k, w in zip( dict_w1[node]['HTLC'], dict_w2[node]['HTLC'], dict_w3[node]['HTLC'], dict_w4[node]['HTLC']) if i == j and j==k and k == w]
    dict_incommon[node]['FEE_BASE'] = [i for i, j, k, w in zip( dict_w1[node]['FEE_BASE'], dict_w2[node]['FEE_BASE'],  dict_w3[node]['FEE_BASE'], dict_w4[node]['FEE_BASE']) if i == j and j==k and k == w]
    dict_incommon[node]['FEE_RATE'] = [i for i, j, k, w in zip( dict_w1[node]['FEE_RATE'], dict_w2[node]['FEE_RATE'],  dict_w3[node]['FEE_RATE'], dict_w4[node]['FEE_RATE']) if i == j and j==k and k == w]

#qua memorizzo le fee che sono state decrementate nel corso delle settimane
dict_min = {}

for node in z:
    dict_min[node] = {'HTLC':[], 'FEE_BASE':[], 'FEE_RATE': []}

    dict_min[node]['HTLC'] = [i for i, j, k, w in zip( dict_w1[node]['HTLC'], dict_w2[node]['HTLC'], dict_w3[node]['HTLC'], dict_w4[node]['HTLC']) if i>j and j>k and k>w]
    dict_min[node]['FEE_BASE'] = [i for i, j, k, w in zip( dict_w1[node]['FEE_BASE'], dict_w2[node]['FEE_BASE'],  dict_w3[node]['FEE_BASE'], dict_w4[node]['FEE_BASE']) if i>j and j>k and k>w]
    dict_min[node]['FEE_RATE'] = [i for i, j, k, w in zip( dict_w1[node]['FEE_RATE'], dict_w2[node]['FEE_RATE'],  dict_w3[node]['FEE_RATE'], dict_w4[node]['FEE_RATE']) if i>j and j>k and k>w]

#qua memorizzo le fee che sono state aumentate nel corso delle settimane
dict_magg = {}

for node in z:
    dict_magg[node] = {'HTLC':[], 'FEE_BASE':[], 'FEE_RATE': []}

    dict_magg[node]['HTLC'] = [i for i, j, k, w in zip( dict_w1[node]['HTLC'], dict_w2[node]['HTLC'], dict_w3[node]['HTLC'], dict_w4[node]['HTLC']) if i<j and j<k and k<w]
    dict_magg[node]['FEE_BASE'] = [i for i, j, k, w in zip( dict_w1[node]['FEE_BASE'], dict_w2[node]['FEE_BASE'],  dict_w3[node]['FEE_BASE'], dict_w4[node]['FEE_BASE']) if i<j and j<k and k<w]
    dict_magg[node]['FEE_RATE'] = [i for i, j, k, w in zip( dict_w1[node]['FEE_RATE'], dict_w2[node]['FEE_RATE'],  dict_w3[node]['FEE_RATE'], dict_w4[node]['FEE_RATE']) if i<j and j<k and k<w]


In [138]:
dict_l = {}
for node in z:
    dict_l[node] = {'TOTALI': int, 'FEE_BASE_EQ': int, 'FEE_RATE_EQ': int}
    dict_l[node]['TOTALI'] = len(dict_w1[node]['FEE_BASE'])
    dict_l[node]['FEE_BASE_EQ'] = len(dict_incommon[node]['FEE_BASE'])
    dict_l[node]['FEE_RATE_EQ'] = len(dict_incommon[node]['FEE_RATE'])

dict_l_min = {}
for node in z:
    dict_l_min[node] = {'TOTALI': int, 'FEE_BASE_EQ': int, 'FEE_RATE_EQ': int}
    dict_l_min[node]['TOTALI'] = len(dict_w1[node]['FEE_BASE'])
    dict_l_min[node]['FEE_BASE_EQ'] = len(dict_min[node]['FEE_BASE'])
    dict_l_min[node]['FEE_RATE_EQ'] = len(dict_min[node]['FEE_RATE'])


In [142]:
i = 0
j = 0
for node in z:
    if(dict_l[node]['FEE_BASE_EQ'] == dict_l[node]['TOTALI']):
        i+=1
    if(dict_l[node]['FEE_RATE_EQ'] == dict_l[node]['TOTALI']):
        j+=1
print(f"nodi che non cambiano le fee_base sui propri canali: {i}, su un totale di {len(z)} nodi --> {i*100/len(z)}%\n")
print(f"nodi che non cambiano le fee_rate sui propri canali: {j}, su un totale di {len(z)} nodi --> {j*100/len(z)}%\n")


i = 0
j = 0
for node in z:
    if(dict_l_min[node]['FEE_BASE_EQ'] == dict_l_min[node]['TOTALI']):
        i+=1
    if(dict_l_min[node]['FEE_RATE_EQ'] == dict_l_min[node]['TOTALI']):
        j+=1
print(f"nodi che non aumentano le fee_base sui propri canali: {i}, su un totale di {len(z)} nodi --> {i*100/len(z)}%\n")
print(f"nodi che non aumentano le fee_rate sui propri canali: {j}, su un totale di {len(z)} nodi --> {j*100/len(z)}%\n")



nodi che non cambiano le fee_base sui propri canali: 7466, su un totale di 7549 nodi --> 98.9005166247185%

nodi che non cambiano le fee_rate sui propri canali: 7425, su un totale di 7549 nodi --> 98.35739833090476%

nodi che non aumentano le fee_base sui propri canali: 0, su un totale di 7549 nodi --> 0.0%

nodi che non aumentano le fee_rate sui propri canali: 0, su un totale di 7549 nodi --> 0.0%



In [26]:
fee_b11 = df1[df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_base_msat'].values.tolist()
fee_b21 = df1[df1['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_base_msat'].values.tolist()
fee_b1 = fee_b11+fee_b21
fee_b12 = df2[df2['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_base_msat'].values.tolist()
fee_b22 = df2[df2['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_base_msat'].values.tolist()
fee_b2 = fee_b12 + fee_b22
fee_b13 = df3[df3['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_base_msat'].values.tolist()
fee_b23 = df3[df3['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_base_msat'].values.tolist()
fee_b3 = fee_b13 + fee_b23
fee_b14 = df4[df4['channel_id'].isin(df_without_dup_copy['channel_id'])]['node1_policy.fee_base_msat'].values.tolist()
fee_b24 = df4[df4['channel_id'].isin(df_without_dup_copy['channel_id'])]['node2_policy.fee_base_msat'].values.tolist()
fee_b4 = fee_b14 + fee_b24


In [4]:
FIGSIZE=(13, 6)
LABELSIZE = 20
TICKSIZE = 18

In [ ]:
# Distribuzione dei nodi e canali nel tempo
fig, ax1 = plt.subplots()
fig.set_figheight(FIGSIZE[1])
fig.set_figwidth(FIGSIZE[0])

color = 'tab:red'
ax1.set_xlabel('Day', fontsize=LABELSIZE)
ax1.set_ylabel('Open channels', fontsize=LABELSIZE)
ax1.set_yscale('log')
ax1.plot(nod, fee_b1, color=color, marker='o')
ax1.tick_params(axis='y', labelcolor=color, labelsize=LABELSIZE)
ax1.tick_params(axis='x', rotation=45, labelsize=LABELSIZE-5)

#ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

#color = 'tab:blue'
#ax2.set_ylabel('Online nodes', fontsize=LABELSIZE)  # we already handled the x-label with ax1
#ax2.set_yscale('log')
#ax2.plot(DATES, num_node_list, color=color, marker='o')
#ax2.tick_params(axis='y', labelcolor=color, labelsize=LABELSIZE)

#fig.tight_layout()  # otherwise the right y-label is slightly clipped

#plt.savefig(os.path.join(DIR  + "monthly" + "distr_nodes_channels_month.png"))

# plt.figure(figsize=FIGSIZE)

# plt.tick_params(axis='x', labelsize=TICKSIZE)
# plt.tick_params(axis='y', labelsize=TICKSIZE)
# plt.xticks(rotation=45)
# plt.xlabel('day', fontsize=LABELSIZE)
# plt.ylabel('channel count', fontsize=LABELSIZE)
# plt.yscale('log')
# plt.plot(DATES, num_node_list, color = "red", marker="o", linestyle="solid")
# plt.plot(DATES, num_open_channels_list, color = "green", marker="o", linestyle="solid")

# plt.tight_layout()

# plt.savefig(os.path.join(DIR  + "monthly" + "distr_nodes_channels_month.png"))



In [35]:
display(len(df1_most_common))

0

In [ ]:
dir_channels = 'datasetAprileMaggio\channels\'

week_1_dates = ["23-04-21", "24-04-21", "25-04-21"] 
week_2_dates = ["26-04-21", "27-04-21", "28-04-21", "29-04-21", "30-04-21", "01-05-21", "02-05-21"]
week_3_dates = ["03-05-21", "04-05-21", "05-05-21", "06-05-21", "07-05-21", "08-05-21", "09-05-21"]
week_4_dates = ["10-05-21"]

with open (dir_dataset+'channels_ln_26-04-21.json') as lun, 
        open (dir_dataset+'channels_ln_27-04-21.json') as mar,
        open (dir_dataset+'channels_ln_28-04-21.json') as mer,
        open (dir_dataset+'channels_ln_29-04-21.json') as gio,
        open (dir_dataset+'channels_ln_30-04-21.json') as ven,
        open (dir_dataset+'channels_ln_01-05-21.json') as sab,
        open (dir_dataset+'channels_ln_02-05-21.json') as dom:

In [ ]:
fee_b11 = df1[df1['channel_id'].isin(df_without_duplicates['channel_id'])]['node1_policy.fee_base_msat'].values.tolist()
